<a href="https://colab.research.google.com/github/macgyver121/DADS7202_final_project/blob/main/prototypeV5_2000d_t5basetot5base_28_12_65.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check GPU

In [1]:
# List all NVIDIA GPUs as avaialble in this computer (or Colab's session)
!nvidia-smi -L

GPU 0: A100-SXM4-40GB (UUID: GPU-6051fa91-9232-cc68-6a97-a162b67724f2)


In [2]:
import sys
print( f"Python {sys.version}\n" )

import numpy as np
print( f"NumPy {np.__version__}\n" )

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
print( f"TensorFlow {tf.__version__}" )
print( f"tf.keras.backend.image_data_format() = {tf.keras.backend.image_data_format()}" )

# Count the number of GPUs as detected by tensorflow
gpus = tf.config.list_physical_devices('GPU')
print( f"TensorFlow detected { len(gpus) } GPU(s):" )
for i, gpu in enumerate(gpus):
  print( f".... GPU No. {i}: Name = {gpu.name} , Type = {gpu.device_type}" )

Python 3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]

NumPy 1.21.6

TensorFlow 2.9.2
tf.keras.backend.image_data_format() = channels_last
TensorFlow detected 1 GPU(s):
.... GPU No. 0: Name = /physical_device:GPU:0 , Type = GPU


Install pytesseract

In [3]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install -q transformers datasets
!pip install -q pytorch-lightning wandb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 2s (2,270 kB/s)
debconf: unable to initi

     |████████████████████████████████| 5.8 MB 31.8 MB/s 
     |████████████████████████████████| 452 kB 99.9 MB/s 
     |████████████████████████████████| 182 kB 98.4 MB/s 
     |████████████████████████████████| 7.6 MB 88.7 MB/s 
     |████████████████████████████████| 132 kB 99.5 MB/s 
     |████████████████████████████████| 212 kB 98.2 MB/s 
     |████████████████████████████████| 127 kB 97.4 MB/s 
     |████████████████████████████████| 800 kB 29.0 MB/s 
     |████████████████████████████████| 1.9 MB 86.3 MB/s 
     |████████████████████████████████| 512 kB 92.3 MB/s 
     |████████████████████████████████| 125 kB 95.4 MB/s 
     |████████████████████████████████| 174 kB 89.4 MB/s 
     |████████████████████████████████| 182 kB 90.2 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 173 kB 96.5 MB/s 
     |████████████████████████████████| 168 kB 90.2 MB/s 
     |████████████████████████████████| 168 kB 76.3 MB/s 
     |██████████

Reset runtime

In [ ]:
import os
os.kill(os.getpid(), 9)

Import library

In [1]:
import cv2
import pytesseract
import shutil
import os
import random
from PIL import Image

import re
import csv
import os

import pyarrow as pa
from datasets import DatasetDict
from datasets.arrow_dataset import Dataset

Mount drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import re
import csv
import os
import pyarrow as pa
import torch
import pyarrow as pa
from datasets import DatasetDict
from datasets.arrow_dataset import Dataset

## Create Data from Chart2text data

In [43]:
List_key = []
# Open the CSV file for reading
for i in range(4000,6000): 
  with open('/content/drive/MyDrive/DADS7202_final/Chart2text/data/'+str(i)+'.csv', 'r') as d:
 # Read the contents of the file into a list of rows
    rows = list(csv.reader(d))
# Concatenate the rows into a single string
    csv_string = '\n'.join(['\t'.join(row) for row in rows])
    csv_string = re.sub('\s+',' ',csv_string)
    List_key.append(csv_string)
len(List_key)

2000

In [44]:
import pickle

with open('List_key_4000to6000.pkl', 'wb') as f:
    pickle.dump(List_key, f)

Load

In [5]:
import pickle

with open('List_key_4000to6000.pkl', 'rb') as f:
    List_key = pickle.load(f)

In [6]:
len(List_key)

2000

In [47]:
txt_key = []
for i in range(4000,6000): 
 f = open('/content/drive/MyDrive/DADS7202_final/Chart2text/captions/'+str(i)+'.txt', 'r')
 content = f.read()
 content = re.sub('\s+',' ',content)
 txt_key.append(content)
len(txt_key)

2000

In [48]:
import pickle

with open('txt_key_4000to6000.pkl', 'wb') as f:
    pickle.dump(txt_key, f)

Load

In [7]:
import pickle

with open('txt_key_4000to6000.pkl', 'rb') as f:
    txt_key = pickle.load(f)

In [8]:
print(List_key[0])

Year Number of births per 1,000 people 2017 16.6 2016 16.8 2015 17.1 2014 17.4 2013 17.6 2012 17.9 2011 18.1 2010 18.3 2009 18.6 2008 18.9 2007 19.2 2006 19.6 2005 20.0


In [9]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
# Create a list of lists that represents the data
train_data = [List_key[:1400] ,txt_key[:1400]]

# Create a list of pyarrow.Array objects from the data
arrays_train = [pa.array(col) for col in train_data]

# Create a pyarrow Table object from the arrays
table1 = pa.Table.from_arrays(arrays_train, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset1 = Dataset(table1)
#print(type(dataset1))

# Create a list of lists that represents the data
val_data = [List_key[1400:1700] ,txt_key[1400:1700]]

# Create a list of pyarrow.Array objects from the data
arrays_val = [pa.array(col) for col in val_data]

# Create a pyarrow Table object from the arrays
table2 = pa.Table.from_arrays(arrays_val, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset2 = Dataset(table2)
print(type(dataset2))

# Create a list of lists that represents the data
test_data = [List_key[1700:] ,txt_key[1700:]]

# Create a list of pyarrow.Array objects from the data
arrays_test = [pa.array(col) for col in test_data]

# Create a pyarrow Table object from the arrays
table3 = pa.Table.from_arrays(arrays_test, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset3 = Dataset(table3)
print(type(dataset3))

# Create a dictionary that maps keys to the Dataset objects
dataset = {
    'train': dataset1,
    'validation': dataset2,
    'test': dataset3
}

# Access a Dataset object using its key
print(dataset)  # Output: Dataset object with train_data
print(type(dataset))
#print(dataset['validation'])  # Output: Dataset object with validation_data
#print(dataset['test'])  # Output: Dataset object with test_data

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
{'train': Dataset({
    features: ['key', 'text'],
    num_rows: 1400
}), 'validation': Dataset({
    features: ['key', 'text'],
    num_rows: 300
}), 'test': Dataset({
    features: ['key', 'text'],
    num_rows: 300
})}
<class 'dict'>


In [11]:
from datasets import DatasetDict

dataset_dict = DatasetDict(dataset)
print(type(dataset_dict))
print(dataset_dict)

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['key', 'text'],
        num_rows: 1400
    })
    validation: Dataset({
        features: ['key', 'text'],
        num_rows: 300
    })
    test: Dataset({
        features: ['key', 'text'],
        num_rows: 300
    })
})


In [12]:
print(dataset['train'][1])

{'key': 'Year Inhabitants in millions 2024* 9.22 2023* 9.17 2022* 9.12 2021* 9.07 2020* 9.01 2019* 8.95 2018 8.89 2017 8.82 2016 8.74 2015 8.63 2014 8.54', 'text': "The statistic shows the total population of Austria from 2014 to 2018 , with projections up until 2024 . In 2018 , the total population of Austria amounted to around 8.89 million inhabitants . See figures for Switzerland 's population and the population of Italy for comparison . "}


In [13]:
example = dataset_dict['train'][1]

print("Key:", example["key"])
print("Text:", example["text"])

Key: Year Inhabitants in millions 2024* 9.22 2023* 9.17 2022* 9.12 2021* 9.07 2020* 9.01 2019* 8.95 2018 8.89 2017 8.82 2016 8.74 2015 8.63 2014 8.54
Text: The statistic shows the total population of Austria from 2014 to 2018 , with projections up until 2024 . In 2018 , the total population of Austria amounted to around 8.89 million inhabitants . See figures for Switzerland 's population and the population of Italy for comparison . 


In [14]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM

*************** เปลี่ยน tokenizer เป็นตาม huggingface เลือก model ****************

In [16]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")  ## edit here

prefix = "Text generation from keyword: "
max_input_length = 512
max_target_length = 256

def preprocess_examples(examples):
  # encode the code-docstring pairs
  codes = examples['key']
  docstrings = examples['text']
  
  inputs = [prefix + code for code in codes]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the summaries
  labels = tokenizer(docstrings, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [17]:
dataset = dataset_dict.map(preprocess_examples, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8)
valid_dataloader = DataLoader(dataset['validation'], batch_size=4)
test_dataloader = DataLoader(dataset['test'], batch_size=4)

In [19]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [20]:
!pip install -q pytorch-lightning wandb

******* ปรับ epoch กับ model เป็นตาม huggingface **********

### Create CodeT5 (model1)

In [21]:
load_directory = '/content/drive/MyDrive/DADS7202_final/model1-t5base-4000d-100e-256-v1'

In [22]:
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl

class CodeT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=100, warmup_steps=100, load_best_model_at_end=True):  ## edit here
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained(load_directory)  ## edit here
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
        
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [23]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [24]:
tokenizer.decode(batch['input_ids'][0])

'Text generation from keyword: Year Percentage of population 2018 13.9% 2017 14% 2016 14.6% 2015 14.8% 2014 15.8% 2013 16% 2012 16.3% 2011 16.4% 2010 15.8% 2009 15.2% 2008 13.4% 2007 13.1% 2006 13.3% 2005 13% 2004 12.5% 2003 12.1% 2002 11.9% 2001 11% 2000 11.1%</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [25]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

"This statistic shows the poverty rate in Ohio from 2000 to 2018. For instance, 13.9 percent of Ohio's population lived below the poverty line in 2018.</s>"

In [26]:
model = CodeT5()

### Train (fit) model1

***** ถ้าใช้ k2t ให้ #logger=wandb_logger ด้วย **********

In [27]:
from transformers import TrainingArguments

In [28]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

import sys
import signal

save_directory = "/content/drive/MyDrive/DADS7202_final/model1-t5basetot5base-2000d-100e-256-v1-timeout"

def save_and_exit(signal, frame):
    model.model.save_pretrained(save_directory)
    sys.exit(0)

signal.signal(signal.SIGINT, save_and_exit)


wandb_logger = WandbLogger(name='t5basetot5base-2000d-100e-256-v1', project='CodeT5')
# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=10,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = Trainer(gpus=1, 
                  default_root_dir="/content/drive/MyDrive/CodeT5/Notebooks/Checkpoints", 
                  logger=wandb_logger,
                  #args = TrainingArguments(load_best_model_at_end=True, output_dir = "/content/drive/MyDrive/Test_output_dir", evaluation_strategy ='epoch', save_steps=200 ),  
                  callbacks=[early_stop_callback, lr_monitor])
trainer.fit(model)

wandb: Currently logged in as: srichaiperksiv (dads7202). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:30

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

#### check layers and weight in model

In [ ]:
# Access the state_dict attribute to get a dictionary of weights
weights = model.state_dict()

# Print the keys and shapes of the weights
#for key, value in weights.items():
    #print(key, value.shape)

In [ ]:
# Get the weights of the first layer
layer_weights = weights['model.decoder.final_layer_norm.weight']

# Print the shape of the weights
#print(layer_weights)

************* ตั้ง path ที่จะ save model อันแรก ***********

In [29]:
save_directory = "/content/drive/MyDrive/DADS7202_final/model1-t5basetot5base-2000d-100e-256-v1"

In [30]:
# save in the current working directory, you can change this of course
model.model.save_pretrained(save_directory)

In [31]:
dataset2 = dataset_dict
print(dataset2['test'])

Dataset({
    features: ['key', 'text'],
    num_rows: 300
})


In [32]:
test_example = dataset2['test'][0]
print("Key:", test_example['key'])

Key: Year Budget balance in trillion yen 2024* -11.67 2023* -10.46 2022* -10.24 2021* -11.12 2020* -12.25 2019* -16.48 2018* -17.64 2017 -17.27 2016 -19.8 2015 -20.23 2014 -28.96


In [33]:
test_example = dataset2['test'][0]
print("Text:", test_example['text'])

Text: The statistic shows the budget balance of Japan from 2014 to 2017 , with projections up until 2024 . In 2017 , the state deficit of Japan was at about 17.27 trillion yen . 


In [ ]:
#from transformers import T5ForConditionalGeneration
#model = T5ForConditionalGeneration.from_pretrained(save_directory)

**************** AutoModelForSeq2SeqLM ของ แต่ละ model ให้ดูตาม huggingface ******************

In [34]:
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)

### Generate data from own model

In [35]:
# prepare for the model
input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
# generate
outputs = model.generate(input_ids,min_length=150, max_length=200)
print("Generated :", tokenizer.decode(outputs[0], skip_special_tokens=True))

Generated : The statistic shows the budget balance in Japan from 2014 to 2017, with projections up until 2024. In 2017, the budget balance in Japan amounted to about 17.27 trillion yen. The forecast predicts a budget surplus of around 11.67 trillion yen in 2024. The forecast predicts a budget surplus of around 1.6 trillion yen in 2024. The forecast predicts a budget surplus of around 1.6 trillion yen in 2024. The budget deficit of Japan is expected to be around 1.7 trillion yen by 2024. The budget deficit of Japan is expected to be around 1.6 trillion yen by 2024. The budget deficit of Japan is expected to be around 1.6 trillion yen by 2024.


Create list of predict data and ref data

In [36]:
predict = []
refer = []
for i in range(300):
  test_example = dataset2['test'][i]
  input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
  outputs = model.generate(input_ids,min_length=150, max_length=200)
  predict.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
  refer.append(test_example['text'])

In [37]:
print(len(predict))
print(len(refer))

300
300


In [ ]:
'''## save list to txt
with open("predict.txt", "w") as output:
    output.write(str(predict))

with open("refer.txt", "w") as output:
    output.write(str(refer))'''

### Evaluation

In [38]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 7.7 MB/s 


In [39]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predict, references=refer)
print(results)

{'bleu': 0.11074332354277122, 'precisions': [0.23540828734449537, 0.12624811605124342, 0.0834123897163457, 0.060672654309191665], 'brevity_penalty': 1.0, 'length_ratio': 2.2116259826230866, 'translation_length': 42764, 'reference_length': 19336}


In [40]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(predict, refer, avg = True)
print(scores)

{'rouge-1': {'r': 0.5068000034149641, 'p': 0.40385252739018807, 'f': 0.43085832536665936}, 'rouge-2': {'r': 0.28189839553799045, 'p': 0.18084985529877828, 'f': 0.2092574325405454}, 'rouge-l': {'r': 0.4925923349656085, 'p': 0.39062738966112087, 'f': 0.41793439225029677}}


## Create data from Ielt DATASET

*********** แก้ path ************

In [ ]:
List_key = []
for i in range(len(os.listdir('/content/drive/MyDrive/DADS7202_final/IELTS/table'))): 
  src = '/content/drive/MyDrive/DADS7202_final/IELTS/table/'+str(i)+'.png'
  img = cv2.imread(src)
  # Adding custom options
  custom_config = r'--psm 1'
  text = pytesseract.image_to_string(img, config=custom_config)
  #print(text)
  text_split = text.split('\n')
  #print(text_split)
  while '' in text_split:
    text_split.remove('')
  while ' ' in text_split:
    text_split.remove(' ')
  key = ' '.join(text_split[:-2])
  List_key.append(key)
len(List_key)

In [ ]:
import pickle

with open('List_key_ielts.pkl', 'wb') as f:
    pickle.dump(List_key, f)

Load ielts key

In [42]:
import pickle

with open('List_key_ielts.pkl', 'rb') as f:
    List_key = pickle.load(f)

In [ ]:
txt_key = []
for i in range(len(os.listdir('/content/drive/MyDrive/DADS7202_final/IELTS/captions'))): 
  f = open('/content/drive/MyDrive/DADS7202_final/IELTS/captions/'+str(i)+'.txt', 'r')
  content = f.read()
  content = re.sub('\s+',' ',content)
  txt_key.append(content)
len(txt_key)

201

In [ ]:
import pickle

with open('txt_key_ielts.pkl', 'wb') as f:
    pickle.dump(txt_key, f)

Load ielts txt

In [43]:
import pickle

with open('txt_key_ielts.pkl', 'rb') as f:
    txt_key = pickle.load(f)

In [44]:
import pyarrow as pa

# Create a list of lists that represents the data
train_data = [List_key[:140] ,txt_key[:140]]

# Create a list of pyarrow.Array objects from the data
arrays_train = [pa.array(col) for col in train_data]

# Create a pyarrow Table object from the arrays
table1 = pa.Table.from_arrays(arrays_train, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset1 = Dataset(table1)
#print(type(dataset1))

# Create a list of lists that represents the data
val_data = [List_key[140:170] ,txt_key[140:170]]

# Create a list of pyarrow.Array objects from the data
arrays_val = [pa.array(col) for col in val_data]

# Create a pyarrow Table object from the arrays
table2 = pa.Table.from_arrays(arrays_val, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset2 = Dataset(table2)
print(type(dataset2))

# Create a list of lists that represents the data
test_data = [List_key[170:] ,txt_key[170:]]

# Create a list of pyarrow.Array objects from the data
arrays_test = [pa.array(col) for col in test_data]

# Create a pyarrow Table object from the arrays
table3 = pa.Table.from_arrays(arrays_test, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset3 = Dataset(table3)
print(type(dataset3))

# Create a dictionary that maps keys to the Dataset objects
ielts_dataset = {
    'train': dataset1,
    'validation': dataset2,
    'test': dataset3
}

# Access a Dataset object using its key
print(ielts_dataset)  # Output: Dataset object with train_data
print(type(ielts_dataset))
#print(dataset['validation'])  # Output: Dataset object with validation_data
#print(dataset['test'])  # Output: Dataset object with test_data

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
{'train': Dataset({
    features: ['key', 'text'],
    num_rows: 140
}), 'validation': Dataset({
    features: ['key', 'text'],
    num_rows: 30
}), 'test': Dataset({
    features: ['key', 'text'],
    num_rows: 31
})}
<class 'dict'>


In [45]:
print(ielts_dataset['train'][0])

{'key': 'Holiday leisure activities Number of hours |At the beach Reading books Chinese tourists 3 8 American tourists 4 5. Turkish tourists 5 4', 'text': 'The bar chart compares the amount of time each day that people from four different countries spent doing activities while on holiday in Greece in August 2019. The activities were going to the beach, visiting different attractions, and reading. Overall, the tourists spent the most time reading books while they spent the least time visiting places. The Chinese tourists spent the longest reading books with a total of eight hours per day. By contrast, the Brazilians spent the longest at the beach with a total of six hours. Both the American and the Turkish tourists spent the same number of hours every day visiting places with a total of three hours each. Similarly, the Turkish and Brazilian groups spent a total of four hours each reading books. The Chinese tourists spent twice the amount of time reading books as the Turkish and Brazilia

In [46]:
from datasets import DatasetDict

dataset_dict = DatasetDict(ielts_dataset)
print(type(dataset_dict))
print(dataset_dict)

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['key', 'text'],
        num_rows: 140
    })
    validation: Dataset({
        features: ['key', 'text'],
        num_rows: 30
    })
    test: Dataset({
        features: ['key', 'text'],
        num_rows: 31
    })
})


**Train ielts data**

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM

*********** แก้ tokenizer ตาม model เรา **********

In [47]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")

prefix = "Text generation from keyword: "
max_input_length = 512
max_target_length = 256

def preprocess_examples(examples):
  # encode the code-docstring pairs
  codes = examples['key']
  docstrings = examples['text']
  
  inputs = [prefix + code for code in codes]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the summaries
  labels = tokenizer(docstrings, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [48]:
dataset = dataset_dict.map(preprocess_examples, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [49]:
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=4)
valid_dataloader = DataLoader(dataset['validation'], batch_size=2)
test_dataloader = DataLoader(dataset['test'], batch_size=2)

In [50]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [ ]:
!pip install -q pytorch-lightning wandb

### Create CodeT5 (model2)

*********** แก้ AutoModelForSeq2SeqLM ตามแต่ละโมเดลตาม huggingface ********

********* แก้ epoch ***********

In [51]:
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl

class CodeT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=100, warmup_steps=100, load_best_model_at_end=True):  #edit here
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)  #edit here
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
        
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [52]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [53]:
tokenizer.decode(batch['input_ids'][0])

'Text generation from keyword: ee BRR Ree Be le el ed Genre Number of % of allreleases % of income from releases ticket sales Comedy 26 17.6 45.4 Horror 14 9.5 20.2 Biopic 1 0.7 9.1 Drama 35 23.6 83 Crime 7 47 47 Action 4 27 41 Documentary 35 23.6 2.9 Thriller B 88 13 Romance 5 34 08 Other 8 5.4 3.2</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [54]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

'The table gives information about films released independently according to genre in the UK and the Republic of Ireland in a single year, 2012. Looking from an overall perspective, it is readily apparent that the vast majority of money was generated by comedy followed closely by horror, then more distantly biopic, and drama, while other genres earned far less. This contrasts with the total number of films in some areas as dramas, documentaries and comedies were most common. Looking at the genres with the greatest disparity first of all, there were 70 documentaries and dramas released though their box office numbers accounted for just 2.9% and 8.3% of sales, respectively. A divergent trend can be seen for comedies and horror in particular with the former constituting 17.6% of all releases and 45.4% of sales and the latter at 9.5% and 20.2% Differences in the remaining genres were more marginal as biopics were.7% of all releases and 9.1% of revenue. The totals for crime were proportiona

Create new model to train again

In [55]:
model = CodeT5()

### Train (fit) model2

In [56]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

wandb_logger = WandbLogger(name='t5basetot5dase-ielts-data', project='CodeT5')
# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=10,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = Trainer(gpus=1, 
                  default_root_dir="/content/drive/MyDrive/CodeT5/Notebooks/Checkpoints", 
                  #logger=wandb_logger, 
                  callbacks=[early_stop_callback, lr_monitor])
trainer.fit(model)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loggers/wandb.py:396: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loops/utilities.py:94: 

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:1595: PossibleUserWarning: The number of training batches (35) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower val

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

******* set new directory ที่จะsaveใน model อันใหม่หลัง train กับ ielts แล้ว ********

In [57]:
save_directory = "/content/drive/MyDrive/DADS7202_final/model2-t5basetot5base-2000d-100e-256-v1"

In [59]:
 # save in the current working directory, you can change this of course
model.model.save_pretrained(save_directory)

In [60]:
dataset2 = dataset_dict
print(dataset2['test'])

Dataset({
    features: ['key', 'text'],
    num_rows: 31
})


In [61]:
test_example = dataset2['test'][1]
print("Key:", test_example['key'])

Key: unemployment rate in Asia 2012 2013 2014 Phippines 5.5 5.3 5.8 China 41 42 43 Japan 3.4 3.3 4.2 Singapore 2 1.8 1.4 India 47 5.2 46


In [62]:
test_example = dataset2['test'][1]
print("Text:", test_example['text'])

Text: In all three years, unemployment was highest in the Philippines. The unemployment rate in the Philippines peaked at nearly 6 percent in 2014. Unemployment was second highest in India for all three years. Third came unemployment in China. Unemployment in Japan was fourth highest, and very close to China’s rate of unemployment in 2014—around 4.25 percent. Singapore had the second to lowest unemployment rates for all three years. Data was not available for Thailand in 2012, but in 2013 and 2014 it had the lowest unemployment rates of all six countries. Unemployment was at its lowest in 2014 in Thailand, when it was below 1.25 percent. For Thailand, India, and Singapore, unemployment was lowest in 2014. For the Philippines, China, and Japan, unemployment was highest in that same year.


******** แก้ AutoModelForSeq2SeqLM ตาม huggingface

### Generate data from ielts model

In [63]:
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)

In [64]:
# prepare for the model
input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
# generate
outputs = model.generate(input_ids,min_length=150, max_length=200)
print("Generated :", tokenizer.decode(outputs[0], skip_special_tokens=True))

Generated : The table shows the unemployment rate in Asia from 2012 to 2014, with projections up until 2014. In 2012, the highest unemployment rate was in the Phippines, with projections up until 2014. In the Philippines, the highest rate was in the Philippines, with projections up until 2014. In terms of the number of people without a job, the number of people without a job rose by a third, to 5.3 in 2014. In the Philippines, the rate was 5.5 in 2013. In the Philippines, the rate was 5.3 in 2013. In the Philippines, the number of people without a job fell by a third, to 3.3 in 2013. In the Philippines, the number of people without a job fell by a third, to 3.3 in 2014. In terms of the number of people without a job, the number of people without a job fell by a third in 2014. In the Philippines, the number of people without a job fell by 


### Evaluation

In [65]:
predict = []
refer = []
for i in range(31):
  test_example = dataset2['test'][i]
  input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
  outputs = model.generate(input_ids,min_length=150, max_length=200)
  predict.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
  refer.append(test_example['text'])

In [66]:
print(len(predict))
print(len(refer))

31
31


In [75]:
i = 11
print(dataset2['test'][i]['key'])
print(predict[i])
print(refer[i])

Percentage of tourists to England who visited Brighton attractions Art Gallery Pavilion Pier Festival 1980 22% 23% 8% 30% 1985 38% 25% 15% 25% 1990 20% 35% 7% 23% 1995 21% 43% 12% 23% 2000 20% 35% 11% 25% 2005 15% 32% 15% 27%
The percentage of tourists who visited Brighton attractions in England in 1980 was much higher than that of the previous two years. In 1980, 22% of tourists visited Brighton attractions, but this figure fell to 23% in 1985 and then to 30% in 1985. The same percentage fell to 30% in 1985 and then again in 1995. In 1985, the Art Gallery Pavilion Pier Festival was the only Brighton attraction to attract more tourists than this. In 1985, 38% of tourists visited Brighton attractions, but this fell to 30%. In 1985, this fell to 30%, but by 1995 this figure had fallen to 15%, and by 2005 this figure had fallen to 23%. In contrast, the proportion of tourists who visited Brighton attractions fell to 23% in 1995, and in 1995 the proportion fell to 23%, and in 1995 the propo

In [68]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=[predict[11]], references=[refer[11]])
print(results)

{'bleu': 0.043419442128165356, 'precisions': [0.4068627450980392, 0.08866995073891626, 0.019801980198019802, 0.004975124378109453], 'brevity_penalty': 1.0, 'length_ratio': 1.1333333333333333, 'translation_length': 204, 'reference_length': 180}


In [ ]:
'''## save list to txt
with open("predict.txt", "w") as output:
    output.write(str(predict))

with open("refer.txt", "w") as output:
    output.write(str(refer))'''

In [ ]:
#pip install evaluate

In [69]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predict, references=refer)
print(results)

{'bleu': 0.045423716809857705, 'precisions': [0.3467509025270758, 0.08277364312942458, 0.022818546914932458, 0.006609142647328804], 'brevity_penalty': 0.9958569815306043, 'length_ratio': 0.9958655401761639, 'translation_length': 5540, 'reference_length': 5563}


In [ ]:
#pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [70]:
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(predict, refer, avg = True)
print(scores)

{'rouge-1': {'r': 0.20620434609182592, 'p': 0.506139776267759, 'f': 0.28716980433797273}, 'rouge-2': {'r': 0.05396375211194664, 'p': 0.13080391505652902, 'f': 0.07443768196698562}, 'rouge-l': {'r': 0.19336394106659618, 'p': 0.4770868155162615, 'f': 0.2695616026940179}}


train with original

In [71]:
model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [72]:
test_example = dataset2['test'][11]
input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
outputs = model.generate(input_ids,min_length=150, max_length=200)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

Brighton attractions Art Gallery Pavilion Pier Festival 1980 22% 23% 8% 30% 1985 38% 25% 15% 25% 1990 20% 35% 7% 23% 1995 21% 43% 12% 23% 2000 20% 35% 11% 25% 2005 15% 32% 35% 11% 25% Brighton attractions Art Gallery Pavilion Pier Festival 1980 22% 23% 8% 30% 1985 38% 25% 15% 25% 1990 20% 35% 7% 23% 1995 21% 43% 12% 23% 8% 30% 1985 38% 25% 15% 15% 25% 1990 20% 35% 7% 23% 1995 2


In [25]:
import cv2
import pytesseract
import shutil
import os
import random
from PIL import Image

import re
import csv

import pyarrow as pa
from datasets import DatasetDict
from datasets.arrow_dataset import Dataset
import torch
import pickle
import transformers
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import wandb

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

import sys
import signal

import evaluate
import rouge
from rouge import Rouge 

In [36]:
print('cv2 :', cv2.__version__)
print('pytesseract :', pytesseract.__version__)
#print('shutil :', shutil.__version__)
#print('os :', os.__version__)
#print('random :', random.__version__)
print('Image :', Image.__version__)
print('re :', re.__version__)
print('csv :', csv.__version__)
print('pyarrow :', pa.__version__)
#print('DatasetDict :', DatasetDict.__version__)
#print('Dataset :', Dataset.__version__)
print('torch :', torch.__version__)
print('pytorch_lightning :', pl.__version__)
#print('pickle :', pickle.__version__)
print('transformers :', transformers.__version__)
print('wandb :', wandb.__version__)
#print('sys :', sys.__version__)
#print('signal :', signal.__version__)
print('evaluate :', evaluate.__version__)
print('rouge :', rouge.__version__)

cv2 : 4.6.0
pytesseract : 0.3.10
Image : 9.3.0
re : 2.2.1
csv : 1.0
pyarrow : 9.0.0
torch : 1.13.0+cu116
pytorch_lightning : 1.8.6
transformers : 4.25.1
wandb : 0.13.7
evaluate : 0.4.0
rouge : 1.0.1
